In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
landrykezebou_vcor_vehicle_color_recognition_dataset_path = kagglehub.dataset_download('landrykezebou/vcor-vehicle-color-recognition-dataset')

print('Data source import complete.')

Data source import complete.


# Dataset Reorganization

In [2]:
import os
import shutil

In [19]:
# Original dataset path (update this)
original_dataset_path = "/kaggle/input/vcor-vehicle-color-recognition-dataset"
original_train = os.path.join(original_dataset_path, "train")
original_val = os.path.join(original_dataset_path, "val")
original_test = os.path.join(original_dataset_path, "test")

In [20]:
# New binary dataset path
binary_train = "/content/binary_dataset/train"
binary_val = "/content/binary_dataset/val"
binary_test = "/content/binary_dataset/test"

In [21]:
# Create folders
for split in [binary_train, binary_val, binary_test]:
    os.makedirs(os.path.join(split, "blue"), exist_ok=True)
    os.makedirs(os.path.join(split, "not_blue"), exist_ok=True)

In [22]:
# Function to move files
def reorganize(original_path, new_path):
    for class_name in os.listdir(original_path):
        class_folder = os.path.join(original_path, class_name)
        if not os.path.isdir(class_folder):
            continue
        target_class = "blue" if class_name.lower() == "blue" else "not_blue"
        for img_file in os.listdir(class_folder):
            src = os.path.join(class_folder, img_file)
            dst = os.path.join(new_path, target_class, img_file)
            shutil.copy(src, dst)  # copy instead of move, so we keep original data

In [23]:
reorganize(original_train, binary_train)
reorganize(original_val, binary_val)
reorganize(original_test, binary_test)

In [24]:
print("✅ Dataset reorganized into 'blue' and 'not_blue'")

✅ Dataset reorganized into 'blue' and 'not_blue'


# Train Binary Classifier

In [25]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models

In [26]:
train_dir = binary_train
val_dir = binary_val

In [27]:
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

In [28]:
train_gen = train_datagen.flow_from_directory(
    train_dir, target_size=(128, 128), batch_size=32, class_mode='binary'
)

val_gen = val_datagen.flow_from_directory(
    val_dir, target_size=(128, 128), batch_size=32, class_mode='binary'
)

Found 7262 images belonging to 2 classes.
Found 1549 images belonging to 2 classes.


In [29]:
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(128,128,3)),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Binary output
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [30]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [31]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=5
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
227/227 ━━━━━━━━━━━━━━━━━━━━ 255s 1s/step - accuracy: 0.9221 - loss: 0.1698 - val_accuracy: 0.9845 - val_loss: 0.0519
Epoch 2/5
227/227 ━━━━━━━━━━━━━━━━━━━━ 249s 1s/step - accuracy: 0.9802 - loss: 0.0610 - val_accuracy: 0.9671 - val_loss: 0.0817
Epoch 3/5
227/227 ━━━━━━━━━━━━━━━━━━━━ 251s 1s/step - accuracy: 0.9832 - loss: 0.0502 - val_accuracy: 0.9852 - val_loss: 0.0506
Epoch 4/5
227/227 ━━━━━━━━━━━━━━━━━━━━ 245s 1s/step - accuracy: 0.9860 - loss: 0.0410 - val_accuracy: 0.9897 - val_loss: 0.0385
Epoch 5/5
227/227 ━━━━━━━━━━━━━━━━━━━━ 251s 1s/step - accuracy: 0.9920 - loss: 0.0277 - val_accuracy: 0.9877 - val_loss: 0.0379


In [32]:
# Save model in Keras format
model.save("blue_car_classifier.keras")

from google.colab import files

# Download Keras format
files.download("blue_car_classifier.keras")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>